# 01.function

In [ ]:
suppressMessages(library(spacexr))
suppressMessages(library(Matrix))
suppressMessages(library(Seurat))
suppressMessages(library(dplyr))
suppressMessages(library(patchwork))
suppressMessages(library(CARD))

plot_all_cell_types <- function(results_df, coords, cell_type_names, outdir,prefixname) {
  barcodes = rownames(results_df[results_df$spot_class != "reject" & results_df$first_type %in% cell_type_names,])
  my_table = coords[barcodes,]
  my_table$class = results_df[barcodes,]$first_type
  n_levels = length(levels(my_table$class))
  my_pal= c('gray','black','blue','purple','#FF7F00','green','red')
  pres = unique(as.integer(my_table$class))
  pres = pres[order(pres)]
  if(n_levels > 21)
    my_pal = pals::polychrome(n_levels)
  if(n_levels > 36)
    stop("Plotting currently supports at most 36 cell types as colors")
   color="black"

  p1 <- ggplot2::ggplot(my_table, ggplot2::aes(x=x, y=y)) + 
    ggplot2::geom_point(ggplot2::aes(size = .15, shape=19,color=class),key_glyph="crossbar") +
    ggplot2::scale_color_manual(values = my_pal[pres])+
    ggplot2::scale_shape_identity() + 
    ggplot2::theme_bw() + 
    ggplot2::coord_fixed()+ggplot2::xlab("")+ggplot2::ylab("")+
    ggplot2::scale_color_manual(values = my_pal[pres])+ 
    ggplot2::scale_shape_identity() + 
    ggplot2::scale_size_identity()+
    ggplot2::coord_fixed()+
    ggplot2::theme(rect = element_rect(colour = "white", fill = "white"),text = element_text(color = "black"),plot.background = element_rect(colour = "white", fill = "white"),panel.background = element_rect(colour = "white", fill = "white"), panel.border = element_blank(),panel.grid.major = element_blank(),panel.grid.minor = element_blank(),legend.key = element_rect(colour = "white", fill = "white"),axis.text=element_blank(),legend.title=element_text(size=30),legend.text=element_text(size=20),legend.key.size=unit(1.2,"inches"),legend.key.height=unit(.5,"inches"),legend.key.width=unit(.5,"inches"))+
    guides(fill=guide_legend( keywidth=0.8, keyheight=0.8,default.unit="inch"))
    ggsave(paste0(outdir,"/",prefixname,".all_cell_types.pdf"),plot=p2,height=20,width=20)
  return(plot)
}

plot_weight <- function(rds,outdir,prefixname){
    norm_weights <- normalize_weights(rds@results$weights)
    Proportion_CARD <- as.matrix(norm_weights)
    ct.visualize <- colnames(Proportion_CARD)
    cellID=rownames(Proportion_CARD)
    x=as.numeric(str_split_fixed(cellID, "[:_]", 3)[,2])
    y=as.numeric(str_split_fixed(cellID, "[:_]", 3)[,3])
    coor=data.frame(x,y)
    rownames(coor) <- cellID
    p2 <- CARD.visualize.prop(proportion = Proportion_CARD,spatial_location = coor,ct.visualize = ct.visualize,colors = c("lightblue","lightyellow","red"),NumCols = 4)+theme(strip.text.x = element_text(size =100, color = "black", face = "bold.italic"))
    ggsave(paste0(outdir,"/",prefixname,".norm_weights.pdf"),plot=p2,height=100,width=200,limitsize = FALSE)
    p3 <- CARD.visualize.Cor(Proportion_CARD,colors = NULL)
    ggsave(paste0(outdir,"/",prefixname,".cor.pdf"),plot=p3,height=6,width=6)
    return(p2)
}


# 02. standard analysis

In [ ]:
args <- commandArgs(TRUE)
scRNA <- args[1]  # Path to the single-cell RNA RDS file
stRNA <- args[2]  # Path to the spatial transcriptomics RDS file
prefix <- args[3] # Prefix for output file
outdir <- args[4] # Output file path

Stdata <- readRDS(stRNA) 
counts <- Stdata@assays$Spatial@counts
coords <- Stdata@meta.data[, c('x','y')]
colsnames <- c("xcoord", "ycoord")
rownames(coords) <- rownames(Stdata@meta.data)
nUMI <- colSums(counts) 
puck <- SpatialRNA(coords, counts, nUMI)


P <- readRDS(scRNA) 
counts <- P[["RNA"]]@counts
P <- AddMetaData(object =P, metadata =P@active.ident, col.name ="seurat_clusters")
meta_data <- P@meta.data
cell_types <- meta_data$seurat_clusters  
names(cell_types) <- colnames(counts) 
cell_types <- as.factor(cell_types) 
nUMI <- colSums(counts)
names(nUMI) <- colnames(counts)
reference <- Reference(counts, cell_types, nUMI)

myRCTD <- create.RCTD(puck, reference, max_cores = 8)
myRCTD <- run.RCTD(myRCTD, doublet_mode = 'doublet')
saveRDS(myRCTD,file=paste0(outdir,prefix,".RCTD.rds"))

norm_weights <- normalize_weights(myRCTD@results$weights)
Proportion_CARD <- as.matrix(norm_weights)
write.table(Proportion_CARD,file=paste(outdir,prefix,".weights.xls",sep=""),sep="\t",quote=F,row.names=T)

results <- myRCTD@results
row.names(results$results_df) <- paste(prefix,row.names(results$results_df),sep="_")
write.csv(results$results_df,file=paste0(outdir,prefix,".results.csv"))
